In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import pandas as pd
import numpy as np

import xgboost as xgb

In [ ]:
train = pd.read_csv('../input/tabular-playground-series-aug-2021/train.csv')
test = pd.read_csv('../input/tabular-playground-series-aug-2021/test.csv')

In [ ]:
train.head()

In [ ]:
X = train.drop(['id','loss'], axis=1)
y = train['loss']

test=test.drop('id',axis=1)

params = {'n_estimators':5000,
          'learning_rate': 0.02,
          'subsample': 0.5,
          'colsample_bytree': 0.7,
          'max_depth': 6,
          'booster': 'gbtree',
          'tree_method': 'gpu_hist',
          'reg_lambda': 60,
          'reg_alpha': 60,
           'n_jobs': 4}

from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import StratifiedKFold, StratifiedShuffleSplit

splits = 12
stf = StratifiedKFold(n_splits=splits, shuffle=True)
oof= np.zeros((X.shape[0],))
prediction = 0
model_fi = 0
total_mean_rmse = 0

for num, (train_id, valid_id) in enumerate(stf.split(X, y)):
    X_train, X_valid = X.loc[train_id], X.loc[valid_id]
    y_train, y_valid = y.loc[train_id], y.loc[valid_id]
    
    model = XGBRegressor(**params)
    model.fit(X_train, y_train,
              eval_set=[(X_train, y_train), (X_valid, y_valid)],
              eval_metric="rmse",verbose=0)
    
    prediction += model.predict(test) / splits
    oof[valid_id] = model.predict(X_valid)
    oof[oof < 0] = 0

    fold_rmse = np.sqrt(mean_squared_error(y_valid, oof[valid_id]))
    print(f"Fold {num} RMSE: {fold_rmse}")

model.feature_importances_

sub=pd.read_csv("../input/tabular-playground-series-aug-2021/sample_submission.csv")
sub["loss"] = prediction

sub.to_csv('submission.csv', index=False)

In [ ]:
import optuna
#optuna.logging.set_verbosity(optuna.logging.WARNING) #suppress message while running

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from xgboost import XGBRegressor

In [ ]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [ ]:
def objective(trial,data=X,target=y):
    
    train_x, test_x, train_y, test_y = train_test_split(data, target, test_size=0.15,random_state=42)
    params = {
        'n_estimators':5000,
        'learning_rate': trial.suggest_categorical('learning_rate', [0.005,0.007,0.009,0.01,0.015,0.02]),
        'subsample': trial.suggest_categorical('subsample', [0.4,0.5,0.6,0.7]),
        'colsample_bytree': trial.suggest_categorical('colsample_bytree', [0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0]),
        'max_depth': trial.suggest_categorical('max_depth', [5,7,9,11,13,15,17,20]),
        'booster': 'gbtree',
        'tree_method': 'gpu_hist', #computer with GPU
#       'tree_method': 'hist',
        'reg_lambda': trial.suggest_int('reg_lambda', 30.0, 60.0),
        'reg_alpha': trial.suggest_int('reg_alpha', 30.0, 60.0),
        'n_jobs': 4}
    model = xgb.XGBRegressor(**params)  
    
    model.fit(train_x,train_y,eval_set=[(test_x,test_y)],early_stopping_rounds=100,verbose=False)
    
    preds = model.predict(test_x)
    
    rmse = mean_squared_error(test_y, preds,squared=False)
    
    return rmse

In [ ]:
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=50, gc_after_trial=True)
print('Number of finished trials:', len(study.trials))
print('Best trial:', study.best_trial.params)

In [ ]:
study.trials_dataframe()

In [ ]:
#plot_optimization_history: shows the scores from all trials as well as the best score so far at each point.
optuna.visualization.plot_optimization_history(study)

#plot_parallel_coordinate: interactively visualizes the hyperparameters and scores
optuna.visualization.plot_parallel_coordinate(study)

#plot_slice: shows the evolution of the search. You can see where in the hyperparameter space your search
# went and which parts of the space were explored more
optuna.visualization.plot_slice(study)

#plot_contour: plots parameter interactions on an interactive chart. 
# You can choose which hyperparameters you would like to explore.
optuna.visualization.plot_contour(study, params=['reg_alpha',
                            #'max_depth',
                            'reg_lambda',
                            'subsample',
                            'learning_rate',
                            'subsample'])

#Visualize parameter importances.
optuna.visualization.plot_param_importances(study)

#Visualize empirical distribution function
optuna.visualization.plot_edf(study)

#Best trial: {'learning_rate': 0, 'subsample': 0, 'colsample_bytree': 1.0, 'max_depth': 7, 'reg_lambda': 0.00487015406161206, 'reg_alpha': 0.001777462879152213}

params = {'n_estimators':5000,
          'learning_rate': 0.02,
#          'subsample': 0.5,
          'colsample_bytree': 1,
          'max_depth': 7,
          'booster': 'gbtree',
          'tree_method': 'gpu_hist',
          'reg_lambda': 0.0487015406161206,
          'reg_alpha': 0.001777462879152213,
           'n_jobs': 4}

splits = 12
stf = StratifiedKFold(n_splits=splits, shuffle=True)
oof= np.zeros((X.shape[0],))
prediction = 0
model_fi = 0
total_mean_rmse = 0

for num, (train_id, valid_id) in enumerate(stf.split(X, y)):
    X_train, X_valid = X.loc[train_id], X.loc[valid_id]
    y_train, y_valid = y.loc[train_id], y.loc[valid_id]
    
    model = XGBRegressor(**params)
    model.fit(X_train, y_train,
              eval_set=[(X_train, y_train), (X_valid, y_valid)],
              eval_metric="rmse",verbose=0)
    
    prediction += model.predict(test) / splits
    oof[valid_id] = model.predict(X_valid)
    oof[oof < 0] = 0

    fold_rmse = np.sqrt(mean_squared_error(y_valid, oof[valid_id]))
    print(f"Fold {num} RMSE: {fold_rmse}")